In [1]:
import numpy as np
import pandas as pd
from docplex.mp.model import Model
import glob
from collections import Counter
import time

from vrp_eval import *
from mat_functs import *

import geopy.distance

In [2]:
def create_adj_mat(mat_dim, routing):
    adj_mat = np.zeros((mat_dim, mat_dim))
    
    for route in routing:
        for i in range(len(route)-1):
            adj_mat[route[i]][route[i+1]] += 1
            
    return adj_mat

In [3]:
def softmax(z):
    return np.exp(z) / np.sum(np.exp(z), axis=1, keepdims=True)

In [4]:
def get_route_1MM(file):
    # returns full route of file
    # stop names; function used in transition matrix construction
    
    df = pd.read_csv(file)
    data = df[df['active']==1][['order','name']]
    
    num_of_routes = len(data[data['order']==1]) + len(data[data['name'].str.contains('SUMY')])
    
    # initialize; fix number of routes
    route_list = []
    for i in range(num_of_routes):
        route_list.append(['SUMY'])

    route_cnt = 0
    for i in range(len(data)):
        if 'SUMY' not in data.iloc[i]['name']:
            if data.iloc[i]['order'] != 1:
                route_list[route_cnt-1].append(data.iloc[i]['name'])
            else:
                route_cnt += 1
                route_list[route_cnt-1].append(data.iloc[i]['name'])
        else:
            route_cnt += 1

    # append 'SUMY' to end of each route
    for i in range(num_of_routes):
#         route_list[i].insert(0,'SUMY')
        route_list[i].append('SUMY')
#         route_list[i].append('SUMY')

    return route_list

In [5]:
# list of all files (train+test)
# file_list = [file for file in sorted(glob.glob("*.csv")) if 'DIMANCHE' in file]
file_list = [file for file in sorted(glob.glob("*.csv"))]

In [6]:
file_list

['180708 DIMANCHE.csv',
 '180709 LUNDI.csv',
 '180710 MARDI.csv',
 '180711 MERCREDI.csv',
 '180712 JEUDI.csv',
 '180713 VENDREDI.csv',
 '180714 SAMEDI.csv',
 '180715 DIMANCHE.csv',
 '180716 LUNDI.csv',
 '180717 MARDI.csv',
 '180718 MERCREDI.csv',
 '180719 JEUDI.csv',
 '180720 VENDREDI.csv',
 '180721 SAMEDI.csv',
 '180722 DIMANCHE.csv',
 '180726 JEUDI.csv',
 '180727 VENDREDI.csv',
 '180728 SAMEDI.csv',
 '180729 DIMANCHE.csv',
 '180730 LUNDI.csv',
 '180731 MARDI.csv',
 '180801 MERCREDI.csv',
 '180802 JEUDI.csv',
 '180803 VENDREDI.csv',
 '180804 SAMEDI.csv',
 '180805 DIMANCHE.csv',
 '180806 LUNDI.csv',
 '180807 MARDI.csv',
 '180808 MERCREDI.csv',
 '180809 JEUDI.csv',
 '180810 VENDREDI.csv',
 '180811 SAMEDI.csv',
 '180812 DIMANCHE.csv',
 '180813 LUNDI.csv',
 '180814 MARDI.csv',
 '180815 MERCREDI.csv',
 '180816 JEUDI.csv',
 '180817 VENDREDI.csv',
 '180818 SAMEDI.csv',
 '180819 DIMANCHE.csv',
 '180820 LUNDI.csv',
 '180821 MARDI.csv',
 '180822 MERCREDI.csv',
 '180823 JEUDI.csv',
 '180824 VEND

In [7]:
# gather all stops visited in historical data
all_stops_set = set()
for instance in file_list:
    all_stops_set.update(set(flatten(get_route_1MM(instance))))
all_stops_list = list(all_stops_set)

all_stops_list.remove('SUMY')
all_stops_list.insert(0, 'SUMY')

all_stops_list

['SUMY',
 'VILLA TER MOLEN (VULPIA)',
 'Parlement Flamand',
 'TFCO KOR',
 'TFCO LIEGE',
 'Dats NATO',
 'TFCO UVO',
 'FARM BASCULE',
 'La Matelotte (Fish4you)',
 'TFCO ANV VOLK',
 'FARM LLN',
 'Dats LEUVEN',
 'Les ferme de nos pilifs',
 'TFCO WAVRE',
 'FARM COOKING',
 'TFCO STOckel',
 'Little Morning',
 'TFCO MA Malines',
 'TFCO WGH',
 'OTAN/ NATO  PROMARITIMES',
 'IVECO EPPEGEM',
 'Residentie Brugsevaart',
 'Actiris',
 'FARM MEISER',
 'DATS LIEGE SERAING',
 'NEW OTAN (Proma)',
 'Sodexo SWIFT',
 'OCEAN MAREE',
 'DATS GRIMBERGEN',
 'Residentie De Linde',
 'TFCO LH Hulpe',
 'TFCO GERPINNES',
 'Residentie Winderkinds',
 'Fish4you',
 'TFCO VDK',
 'Dats Waterloo',
 'TFCO AUD',
 'DATS MELLE',
 'FARM ST CATH',
 'TRAVIE (SEL)',
 'Les idees a la pele',
 'Sodexo SWIFT SOLVAY',
 'TFCO AAL',
 'Dats MALINES',
 'BIA MARA ANVERS',
 'TFCO OVErijse',
 'Dats Anderlecht',
 'FARM FC-IXL',
 'BON PLAISIR',
 'Carfrig',
 'DATS GOSSELIES',
 'TFCO WOO Waterloo',
 'TFCO ANV BRASSCHAT',
 'Sodexo EUROCONTROL',
 'TF

In [8]:
coords_list = [('SUMY', 50.911043, 4.403273)]

for item in all_stops_list[1:]:
    for file in file_list:
        df = pd.read_csv(file)
        data = df[df['active']==1][['order','name', 'lat', 'lng']]
        
        if item in data.values:
            coords = (item, data[data['name'] == item]['lat'].values[0], data[data['name'] == item]['lng'].values[0])
            coords_list.append(coords)
            break

coords_list

[('SUMY', 50.911043, 4.403273),
 ('VILLA TER MOLEN (VULPIA)', 51.33302, 4.36675),
 ('Parlement Flamand', 50.848518, 4.368233),
 ('TFCO KOR', 50.81933, 3.26903),
 ('TFCO LIEGE', 50.62798, 5.56695),
 ('Dats NATO', 50.89316, 4.43101),
 ('TFCO UVO', 50.7946, 4.37342),
 ('FARM BASCULE', 50.814585, 4.365606),
 ('La Matelotte (Fish4you)', 50.81378, 4.3485699),
 ('TFCO ANV VOLK', 51.21187, 4.39655),
 ('FARM LLN', 50.6689, 4.61494),
 ('Dats LEUVEN', 50.87273, 4.68519),
 ('Les ferme de nos pilifs', 50.90368, 4.38881),
 ('TFCO WAVRE', 50.71678, 4.61559),
 ('FARM COOKING', 50.80935, 4.3949999),
 ('TFCO STOckel', 50.8388599, 4.46446),
 ('Little Morning', 50.63115, 4.53516),
 ('TFCO MA Malines', 51.02265, 4.47407),
 ('TFCO WGH', 50.84471, 4.41684),
 ('OTAN/ NATO  PROMARITIMES', 50.877383, 4.429926),
 ('IVECO EPPEGEM', 50.9688299, 4.45545),
 ('Residentie Brugsevaart', 51.0711299, 3.69216),
 ('Actiris', 50.85132, 4.37023),
 ('FARM MEISER', 50.85543, 4.3951399),
 ('DATS LIEGE SERAING', 50.60829, 5.5184

In [9]:
# create full distance matrix
mat_dim = len(all_stops_list)
full_dist_mat = np.zeros((mat_dim, mat_dim))

for i in range(mat_dim):
    for j in range(mat_dim):
        full_dist_mat[i][j] = geopy.distance.distance((coords_list[i][1], coords_list[i][2]), 
                                                      (coords_list[j][1], coords_list[j][2])).km
        
full_dist_mat[0]

array([ 0.        , 47.01478839,  7.37988244, 80.50007195, 87.92243261,
        2.78654697, 13.12322912, 11.05337205, 11.48509099, 33.47015775,
       30.79564623, 20.28860498,  1.30606877, 26.28563587, 11.32788492,
        9.11207395, 32.49663508, 13.37502776,  7.44079978,  4.18785538,
        7.40116774, 53.00900652,  7.03915979,  6.213145  , 85.58757301,
        3.26075935, 20.33791955,  9.84940372,  1.9667372 , 11.49464936,
       20.72162969, 63.30338787, 14.492475  , 11.48509099, 11.21484719,
       25.10545093, 10.08494792, 45.85722064,  7.55478867, 10.29440379,
        5.64585265, 19.70509673, 25.70380412, 15.76958651, 34.47087225,
       15.44084235, 11.63298407,  8.94983349,  5.83194266, 13.1830451 ,
       49.25859064, 21.18738224, 42.47059874,  3.94225541, 10.26839292,
       46.07378917,  7.63783798,  8.9642191 ,  9.74359101, 39.77617876,
        8.49130532,  6.91316688, 49.87273957, 20.33791955,  7.77939698,
       10.4084372 , 21.15261411,  1.30606877, 15.92224246, 16.67

In [10]:
# create full proba mat

def create_full_proba_mat(all_stops_list, train):
    
    mat_dim = len(all_stops_list)
    full_proba_mat = np.zeros((mat_dim, mat_dim))

    # collect historical data
    data = [] # list of route plans
    for instance in train:
        data.append(get_route_1MM(instance))

    # construct 3D matrix
    for (hist_ind, plan) in enumerate(data):
        
        weight = 0.7*(1-0.7)**(len(data)-(hist_ind+1)) # exponential weighing
    
        for route in plan:
            for i in range(len(route)-1):
                full_proba_mat[all_stops_list.index(route[i])][all_stops_list.index(route[i+1])] += weight
                
    return(full_proba_mat)

### Train

In [11]:
n_test = 49
w_d = 1
w_p = 1

In [12]:
# initialize df of results
train_df = pd.DataFrame()
train_df = pd.DataFrame(columns = ['Date','Target','Solution', 'Status', 'SD','%SD','AD','%AD', 'Gap', 'Time', 'w_d', 'w_p', 'Epoch'])

for epoch in range(4):

    for current_inst in file_list[:-n_test]:
#     for current_inst in file_list[:5]:

        actual = convert(get_route_1MM(current_inst), all_stops_list)

        # relevant stops of the current instance
        current_idx_list = list(set(flatten(actual)))

        if file_list.index(current_inst) != 0:
            train = file_list[:file_list.index(current_inst)]
        else:
            train = [file_list[0]]

        # create full proba mat
        full_proba_mat = create_full_proba_mat(all_stops_list, train)

        # proba mat for current instance
        init_proba_mat = full_proba_mat[np.ix_(current_idx_list, current_idx_list)]

        # proba mat transformations
        init_proba_mat = init_proba_mat + 1 # smoothing
        init_proba_mat = init_proba_mat/init_proba_mat.sum(axis=1, keepdims=True) # normalize
        init_proba_mat = -np.log(init_proba_mat) # negative log

        # dist mat for current instance
        init_dist_mat = full_dist_mat[np.ix_(current_idx_list, current_idx_list)]

        # dist mat transformations
        init_dist_mat = init_dist_mat + 1 # smoothing
        init_dist_mat = 1/init_dist_mat # invert
        init_dist_mat = -np.log(init_dist_mat) # negative log

#         init_dist_mat = softmax(init_dist_mat)

        n = len(set(flatten(actual)))-1 # len(stop_list)-1

        N = [i for i in range(1, n+1)]
        rt_count = len(actual)

        Q = n

        V = [0] + N
        #         q = dict(zip(V, cap))
        q = {i : 1 for i in N} # dictionary of demands

        # create set of arcs
        A = [(i,j) for i in V for j in V if i!=j]

        # solve using CPLEX
        mdl = Model('CVRP')

        #         # gap tolerance
        #         mdl.parameters.mip.tolerances.mipgap=0.02;
        #         mdl.parameters.emphasis.mip=3;
        #         mdl.parameters.mip.strategy.search=1;

        x = mdl.binary_var_dict(A, name='x')
        u = mdl.continuous_var_dict(N, ub=Q, name='u')

        proba_matrix = w_d*init_dist_mat + w_p*init_proba_mat

        mdl.minimize(mdl.sum((proba_matrix[i,j])*x[i,j] for i,j in A))

        # constraints
        mdl.add_constraints(mdl.sum(x[i,j] for j in V if j!=i)==1 for i in N)
        mdl.add_constraints(mdl.sum(x[i,j] for i in V if i!=j)==1 for j in N)
        mdl.add_indicator_constraints(mdl.indicator_constraint(x[i,j], u[i]+q[j]==u[j]) for i,j in A if i!=0 and j!=0)
        mdl.add_constraints(u[i]>=q[i] for i in N)

        # fix number of routes
        mdl.add_constraints(mdl.sum(x[0,j] for j in N)<=rt_count for j in N)
        mdl.add_constraints(mdl.sum(x[0,j] for j in N)>=rt_count for j in N)

        # set time limit
        #         mdl.parameters.timelimit = 3600*12 # 5 hours (86400 - 1 day)
        mdl.parameters.timelimit = 20 # 5 hours (86400 - 1 day)

        start = time.time()

        # set (log_output=True) to display solution
        solution = mdl.solve()
        #         solution = mdl.solve(log_output=True)
        #         print("gap:",mdl.MIP.get_mip_relative_gap())
        #         print("gap:",mdl.get_solve_details().mip_relative_gap)
        gap = mdl.get_solve_details().mip_relative_gap

        end = time.time()

        sol_time = end-start

    #     print("solution time:", sol_time)

    #     print('mdl.objective_value =', mdl.objective_value)

        active_arcs = [a for a in A if x[a].solution_value>0.9]
        start_arcs = [a for a in active_arcs if a[0]==0]

        res_plan = []
        for arc in start_arcs:    
            res_route = [0,arc[1]]
            while res_route[-1]!=0:
                for a in active_arcs:
                    if res_route[-1]==a[0]:
                        res_route.append(a[1])
            res_plan.append(res_route)

        d = dict(zip(current_idx_list, V))

        new_actual = []

        for route in actual:
            new_route = []
            for s in route:
                new_route.append(d[s])
            new_actual.append(new_route)

        actual_for_comp = [route for route in new_actual]
        
        train_df.loc[len(train_df)] = [current_inst.split(" ")[0], 
                                           actual_for_comp, 
                                           res_plan, gap,
                                           eval_sd(actual_for_comp,res_plan)[1], 
                                           eval_sd(actual_for_comp,res_plan)[2],
                                           eval_ad(actual_for_comp,res_plan)[1], 
                                           eval_ad(actual_for_comp,res_plan)[2], 
                                           gap, sol_time, w_d, w_p, epoch]

        mat_dim = len(init_dist_mat)

        # update weights

        pred_adj = create_adj_mat(mat_dim, res_plan)
        act_adj = create_adj_mat(mat_dim, actual_for_comp)

        w_d = w_d + 0.0002*(np.multiply(pred_adj, init_dist_mat).sum() - np.multiply(act_adj, init_dist_mat).sum()) 
        w_p = w_p + 0.0002*(np.multiply(pred_adj, init_proba_mat).sum() - np.multiply(act_adj, init_proba_mat).sum()) 

        print(w_d, w_p, end=' ')

0.9987793631835802 0.9996348938259876 0.9965985467124366 0.9986132874241559 0.9945552066439695 0.99658740337862 0.9924416942168488 0.9964605326908279 0.9908389495714652 0.9947345057829757 0.9885109840481776 0.9924442946874037 0.9865980487582309 0.9888883796015386 0.9858575317711449 0.9868424487095001 0.9816551486256044 0.9839096104042689 0.9790870051894768 0.9811331295192457 0.9776078791212043 0.9813803220082503 0.9764471845043717 0.9813924427973444 0.9753873131742562 0.9818771291799595 0.9746874841786342 0.9806130075483486 0.9741114180140741 0.9785211551444689 0.9714664639846522 0.977081896581656 0.969780328168563 0.9779016423598594 0.9682743644625545 0.9782086321959291 0.9677792858481838 0.9765998353202704 0.9646870517221323 0.9741790519058323 0.9617827700007017 0.9703976881440554 0.9592743558958311 0.9689214374472435 0.9572587139273193 0.9670739046881345 0.9560969237576795 0.9671627108424156 0.954394193035323 0.967202795898015 0.9538991144209523 0.9654668304240229 0.9526949540954873

0.5691472244757888 0.6172372316752085 0.5656932738189662 0.6139686397615104 0.5635872166025689 0.6129680430886142 0.561676030769045 0.6114636933828841 0.5594059233513932 0.6102075660400025 0.559689495456357 0.6081242028976226 0.558005792145911 0.6070138791646272 0.5536869492988378 0.6023297357185459 0.5503552790593152 0.5990629101822664 0.5455915269136821 0.5958112388414535 0.5432693166240655 0.5919917227291702 0.5408682280191144 0.5901040421601209 0.5396075978445597 0.5898645356203202 0.5381074037042464 0.5890526423919018 0.5345590575439192 0.584864560194538 0.5327058757569199 0.582560382054478 0.5310363854666786 0.581429234299395 0.5295652586525063 0.5804955599682486 0.5272761079165179 0.5786123760031966 0.5257230677748708 0.5772443427620815 0.5241529109473586 0.5762972208067604 0.5231653134075182 0.5751746166174113 0.5210863965420947 0.5735237692039064 0.5182975661831687 0.5708927708472571 0.5158641677109893 0.5676690234979679 0.5128438322334066 0.5648203533037743 0.5105876266734537

0.15771345478195756 0.20311297987602167 0.15484157675378699 0.20048904008467236 0.1538973244810703 0.19975137438941584 0.15405637604065245 0.19876094484286344 0.15365338737927833 0.19860731847396504 0.15266909595128797 0.19650932971950058 0.15242605106836052 0.19639590843040894 0.1503877594379773 0.195100733025516 0.1493284811893 0.19295101804071707 0.14849759205575433 0.19252350496123763 0.14593267298149798 0.19151124498529287 0.14496500576043636 0.18916847901593578 0.1432582853167329 0.18690337023318745 0.1423435855261191 0.1851296941025286 0.14160480459771801 0.1844830521648564 0.14129132164339206 0.18353185066662026 0.13924650644292333 0.18136484204591344 0.1378774871071508 0.18015630755377876 0.13591880363563444 0.17865961692591936 0.1347125542743484 0.17749742830345777 0.13468740138309293 0.17689814458727116 0.1307769727025285 0.17532408199866362 0.1284349016587892 0.1740542023543159 0.1271849978223569 0.1732676096716457 0.12639058985248075 0.17213084446619517 0.12547746363054865

In [ ]:
train_df

In [13]:
train_df.to_csv('mixing_train.csv')

In [ ]:
init_dist_mat[0]

In [ ]:
init_proba_mat[0]

### Test

In [ ]:
# w_d_trained = w_d/(w_d + w_p)
# w_p_trained = w_p/(w_d + w_p)

In [ ]:
# w_d_trained, w_p_trained

In [ ]:
# initialize df of results
results_df = pd.DataFrame()
results_df = pd.DataFrame(columns = ['Date','Target','Solution', 'Status', 'SD','%SD','AD','%AD', 'Gap', 'Time', 'w_d', 'w_p'])

for current_inst in file_list[-n_test:]:
    
    actual = convert(get_route_1MM(current_inst), all_stops_list)

    # relevant stops of the current instance
    current_idx_list = list(set(flatten(actual)))
    
    train = file_list[:file_list.index(current_inst)]
    
    # create full proba mat
    full_proba_mat = create_full_proba_mat(all_stops_list, train)
    
    # proba mat for current instance
    init_proba_mat = full_proba_mat[np.ix_(current_idx_list, current_idx_list)]

    # proba mat transformations
    init_proba_mat = init_proba_mat + 1 # smoothing
    init_proba_mat = init_proba_mat/init_proba_mat.sum(axis=1, keepdims=True) # normalize
    init_proba_mat = -np.log(init_proba_mat) # negative log

    # dist mat for current instance
    init_dist_mat = full_dist_mat[np.ix_(current_idx_list, current_idx_list)]

    # dist mat transformations
    init_dist_mat = init_dist_mat + 1 # smoothing
    init_dist_mat = 1/init_dist_mat # invert
    init_dist_mat = -np.log(init_dist_mat) # negative log

#     init_dist_mat = softmax(init_dist_mat)
    
    n = len(set(flatten(actual)))-1 # len(stop_list)-1

    N = [i for i in range(1, n+1)]
    rt_count = len(actual)

    Q = n

    V = [0] + N
    #         q = dict(zip(V, cap))
    q = {i : 1 for i in N} # dictionary of demands

    # create set of arcs
    A = [(i,j) for i in V for j in V if i!=j]

    # solve using CPLEX
    mdl = Model('CVRP')

    #         # gap tolerance
    #         mdl.parameters.mip.tolerances.mipgap=0.02;
    #         mdl.parameters.emphasis.mip=3;
    #         mdl.parameters.mip.strategy.search=1;

    x = mdl.binary_var_dict(A, name='x')
    u = mdl.continuous_var_dict(N, ub=Q, name='u')

#     proba_matrix = w_d_trained*init_dist_mat + w_p_trained*init_proba_mat
    proba_matrix = w_d*init_dist_mat + w_p*init_proba_mat

    mdl.minimize(mdl.sum((proba_matrix[i,j])*x[i,j] for i,j in A))

    # constraints
    mdl.add_constraints(mdl.sum(x[i,j] for j in V if j!=i)==1 for i in N)
    mdl.add_constraints(mdl.sum(x[i,j] for i in V if i!=j)==1 for j in N)
    mdl.add_indicator_constraints(mdl.indicator_constraint(x[i,j], u[i]+q[j]==u[j]) for i,j in A if i!=0 and j!=0)
    mdl.add_constraints(u[i]>=q[i] for i in N)

    # fix number of routes
    mdl.add_constraints(mdl.sum(x[0,j] for j in N)<=rt_count for j in N)
    mdl.add_constraints(mdl.sum(x[0,j] for j in N)>=rt_count for j in N)

    # set time limit
    #         mdl.parameters.timelimit = 3600*12 # 5 hours (86400 - 1 day)
    mdl.parameters.timelimit = 20 # 5 hours (86400 - 1 day)

    start = time.time()

    # set (log_output=True) to display solution
    solution = mdl.solve()
    #         solution = mdl.solve(log_output=True)
    #         print("gap:",mdl.MIP.get_mip_relative_gap())
    #         print("gap:",mdl.get_solve_details().mip_relative_gap)
    gap = mdl.get_solve_details().mip_relative_gap

    end = time.time()

    sol_time = end-start

    print("solution time:", sol_time)

    print('mdl.objective_value =', mdl.objective_value)
    
    active_arcs = [a for a in A if x[a].solution_value>0.9]
    start_arcs = [a for a in active_arcs if a[0]==0]

    res_plan = []
    for arc in start_arcs:    
        res_route = [0,arc[1]]
        while res_route[-1]!=0:
            for a in active_arcs:
                if res_route[-1]==a[0]:
                    res_route.append(a[1])
        res_plan.append(res_route)

    d = dict(zip(current_idx_list, V))

    new_actual = []

    for route in actual:
        new_route = []
        for s in route:
            new_route.append(d[s])
        new_actual.append(new_route)

    actual_for_comp = [route for route in new_actual]
    
    results_df.loc[len(results_df)] = [current_inst.split(" ")[0], 
                                       actual_for_comp, 
                                       res_plan, gap,
                                       eval_sd(actual_for_comp,res_plan)[1], 
                                       eval_sd(actual_for_comp,res_plan)[2],
                                       eval_ad(actual_for_comp,res_plan)[1], 
                                       eval_ad(actual_for_comp,res_plan)[2], 
                                       gap, sol_time, w_d, w_p]
    
    mat_dim = len(init_dist_mat)

    # update weights

    pred_adj = create_adj_mat(mat_dim, res_plan)
    act_adj = create_adj_mat(mat_dim, actual_for_comp)

#     w_d_trained = w_d_trained + 0.0002*(np.multiply(pred_adj, init_dist_mat).sum() - np.multiply(act_adj, init_dist_mat).sum()) 
#     w_p_trained = w_p_trained + 0.0002*(np.multiply(pred_adj, init_proba_mat).sum() - np.multiply(act_adj, init_proba_mat).sum())
    
    w_d = w_d + 0.0002*(np.multiply(pred_adj, init_dist_mat).sum() - np.multiply(act_adj, init_dist_mat).sum()) 
    w_p = w_p + 0.0002*(np.multiply(pred_adj, init_proba_mat).sum() - np.multiply(act_adj, init_proba_mat).sum()) 

In [ ]:
results_df

In [ ]:
results_df.mean()

In [ ]:
results_df.to_csv('SOP_Mixing_ALL_nosoft_0002_4ep.csv')

In [ ]:
current_inst

In [ ]:
actual = convert(get_route(current_inst), all_stops_list)

# relevant stops of the current instance
current_idx_list = list(set(flatten(actual)))

In [ ]:
# proba mat for current instance

init_proba_mat = full_proba_mat[np.ix_(current_idx_list, current_idx_list)]

In [ ]:
init_proba_mat[0]

In [ ]:
# proba mat transformations

init_proba_mat = init_proba_mat + 1 # smoothing
init_proba_mat = init_proba_mat/init_proba_mat.sum(axis=1, keepdims=True) # normalize
init_proba_mat = -np.log(init_proba_mat) # negative log

In [ ]:
init_proba_mat[0]

In [ ]:
# dist mat for current instance

init_dist_mat = full_dist_mat[np.ix_(current_idx_list, current_idx_list)]

In [ ]:
init_dist_mat[0]

In [ ]:
# dist mat transformations

init_dist_mat = init_dist_mat + 1 # smoothing
init_dist_mat = 1/init_dist_mat # invert
init_dist_mat = -np.log(init_dist_mat) # negative log

In [ ]:
init_dist_mat[0]

In [ ]:
actual = convert(get_route(current_inst), all_stops_list)

In [ ]:
actual

In [ ]:
n = len(set(flatten(actual)))-1 # len(stop_list)-1

In [ ]:
n

In [ ]:
N = [i for i in range(1, n+1)]
rt_count = len(actual)

Q = n

In [ ]:
V = [0] + N
#         q = dict(zip(V, cap))
q = {i : 1 for i in N} # dictionary of demands

# create set of arcs
A = [(i,j) for i in V for j in V if i!=j]

# solve using CPLEX
mdl = Model('CVRP')

#         # gap tolerance
#         mdl.parameters.mip.tolerances.mipgap=0.02;
#         mdl.parameters.emphasis.mip=3;
#         mdl.parameters.mip.strategy.search=1;

x = mdl.binary_var_dict(A, name='x')
u = mdl.continuous_var_dict(N, ub=Q, name='u')

In [ ]:
w_d = 1
w_p = 1

In [ ]:
proba_matrix = w_d*init_dist_mat + w_p*init_proba_mat 

In [ ]:
mdl.minimize(mdl.sum((proba_matrix[i,j])*x[i,j] for i,j in A))

In [ ]:
# constraints
mdl.add_constraints(mdl.sum(x[i,j] for j in V if j!=i)==1 for i in N)
mdl.add_constraints(mdl.sum(x[i,j] for i in V if i!=j)==1 for j in N)
mdl.add_indicator_constraints(mdl.indicator_constraint(x[i,j], u[i]+q[j]==u[j]) for i,j in A if i!=0 and j!=0)
mdl.add_constraints(u[i]>=q[i] for i in N)

# fix number of routes
mdl.add_constraints(mdl.sum(x[0,j] for j in N)<=rt_count for j in N)
mdl.add_constraints(mdl.sum(x[0,j] for j in N)>=rt_count for j in N)

# set time limit
#         mdl.parameters.timelimit = 3600*12 # 5 hours (86400 - 1 day)
mdl.parameters.timelimit = 20 # 5 hours (86400 - 1 day)

start = time.time()

# set (log_output=True) to display solution
solution = mdl.solve()
#         solution = mdl.solve(log_output=True)
#         print("gap:",mdl.MIP.get_mip_relative_gap())
#         print("gap:",mdl.get_solve_details().mip_relative_gap)
gap = mdl.get_solve_details().mip_relative_gap

end = time.time()

sol_time = end-start

print("solution time:", sol_time)

print('mdl.objective_value =', mdl.objective_value)

In [ ]:
active_arcs = [a for a in A if x[a].solution_value>0.9]
start_arcs = [a for a in active_arcs if a[0]==0]

In [ ]:
res_plan = []
for arc in start_arcs:    
    res_route = [0,arc[1]]
    while res_route[-1]!=0:
        for a in active_arcs:
            if res_route[-1]==a[0]:
                res_route.append(a[1])
    res_plan.append(res_route)

In [ ]:
res_plan

In [ ]:
actual

In [ ]:
d = dict(zip(current_idx_list, V))
#         print(idx_list)

new_actual = []

for route in actual:
    new_route = []
    for s in route:
        new_route.append(d[s])
    new_actual.append(new_route)
    
actual_for_comp = [route[1:-1] for route in new_actual]

In [ ]:
actual_for_comp

In [ ]:
eval_ad(actual_for_comp, res_plan)

In [ ]:
eval_sd(actual_for_comp, res_plan)

In [ ]:
actual_for_comp

In [ ]:
mat_dim = len(init_dist_mat)

In [ ]:
pred_adj = create_adj_mat(mat_dim, res_plan)
act_adj = create_adj_mat(mat_dim, actual_for_comp)

In [ ]:
w_d = w_d + 0.00001*(np.multiply(pred_adj, init_dist_mat).sum() - np.multiply(act_adj, init_dist_mat).sum()) 

In [ ]:
w_p = w_p + 0.00001*(np.multiply(pred_adj, init_proba_mat).sum() - np.multiply(act_adj, init_proba_mat).sum()) 

In [ ]:
def softmax(z):
    return np.exp(z) / np.sum(np.exp(z), axis=1, keepdims=True)

In [ ]:
mat = [[1, 0, 0],
       [1, 1, 1]
      ]

In [ ]:
mat

In [ ]:
softmax(mat)